# Global Texture Descriptor

### 分形维数 Fractal dimension (Hausdorff维数)
- 1维

In [86]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

In [87]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [88]:
# print(mpimg.imread("./pkq.png").shape)
image = cv2.imread("./pkq.png")
image_gray = rgb2gray(image)

# finding all the non-zero pixels
pixels=[]
for i in range(image_gray.shape[0]):
    for j in range(image_gray.shape[1]):
        if image_gray[i,j] > 0:
            pixels.append((i,j))

Lx = image_gray.shape[1]
Ly= image_gray.shape[0]
# print(Lx, Ly)
pixels = np.array(pixels)
# print(pixels.shape)

# computing the fractal dimension
#considering only scales in a logarithmic list
scales = np.logspace(0.01, 1, num=10, endpoint=False, base=2)
Ns = []
# looping over several scales
for scale in scales:
    # print ("======= Scale :",scale)
    # computing the histogram
    H, edges = np.histogramdd(pixels, bins=(np.arange(0,Lx,scale),np.arange(0,Ly,scale)))
    Ns.append(np.sum(H>0))

# linear fit, polynomial of degree 1
coeffs = np.polyfit(np.log(scales), np.log(Ns), 1)

# plt.plot(np.log(scales),np.log(Ns), 'o', mfc='none')
# plt.plot(np.log(scales), np.polyval(coeffs,np.log(scales)))
# plt.xlabel('log $\epsilon$')
# plt.ylabel('log N')
# plt.savefig('sierpinski_dimension.pdf')

Hausdorff_D = -coeffs[0]
print ("The Hausdorff dimension is", Hausdorff_D) #the fractal dimension is the OPPOSITE of the fitting coefficient
# np.savetxt("scaling.txt", list(zip(scales,Ns)))

The Hausdorff dimension is 1.9662861686967643


### 粒度 Coarseness
- 1维

In [89]:
# image = rgb2gray(cv2.imread("./pkq.png"))
# image = cv2.imread("./pkq.png")
ar = image[:,:,0].flatten()
ag = image[:,:,1].flatten()
ab = image[:,:,2].flatten()

histogram_r = np.histogram(ar, bins=255, density=True)
histogram_g = np.histogram(ag, bins=255, density=True)
histogram_b = np.histogram(ab, bins=255, density=True)

In [90]:
Sm_r = np.mean(histogram_r[0])
Sd_r = 0
for i in range(0, 255):
    Sd_r += pow(histogram_r[0][i] - Sm_r, 2)

Sm_g = np.mean(histogram_g[0])
Sd_g = 0
for i in range(0, 255):
    Sd_g += pow(histogram_g[0][i] - Sm_g, 2)

Sm_b = np.mean(histogram_b[0])
Sd_b = 0
for i in range(0, 255):
    Sd_b += pow(histogram_b[0][i] - Sm_b, 2)

Sd = Sd_r + Sd_g + Sd_b
C = 1 - 1 / (1 + Sd)
print ("The Coarseness is", C)

The Coarseness is 0.5984062051758858


### 熵 Entropy
- 1维

In [91]:
Se_r = 0
for i in range(0, 255):
    Se_r += histogram_r[0][i] * math.log(histogram_r[0][i] + 0.00001, 2)
print(Se_r)
Se_g = 0
for i in range(0, 255):
    Se_g += histogram_g[0][i] * math.log(histogram_g[0][i] + 0.00001, 2)

Se_b = 0
for i in range(0, 255):
    Se_b += histogram_b[0][i] * math.log(histogram_b[0][i] + 0.00001, 2)

Se = -(Se_r + Se_g + Se_b)
print ("The Entropy is", Se)

-1.5391217507548978
The Entropy is 5.054648392802223


### 空间灰度差统计 spatial gray-level difference statistics
- 维数与图像有关（相同）

In [93]:
def fast_glcm(img, vmin=0, vmax=255, nbit=8, kernel_size=5):
    mi, ma = vmin, vmax
    ks = kernel_size
    h,w = img.shape

    # digitize
    bins = np.linspace(mi, ma+1, nbit+1)
    gl1 = np.digitize(img, bins) - 1
    gl2 = np.append(gl1[:,1:], gl1[:,-1:], axis=1)

    # make glcm
    glcm = np.zeros((nbit, nbit, h, w), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1

    kernel = np.ones((ks, ks), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)

    glcm = glcm.astype(np.float32)
    return glcm

def fast_glcm_mean(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm mean
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    mean = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            mean += glcm[i,j] * i / (nbit)**2

    return mean
#
# def fast_glcm_std(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm std
#     '''
#     h,w = img.shape
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     mean = np.zeros((h,w), dtype=np.float32)
#     for i in range(nbit):
#         for j in range(nbit):
#             mean += glcm[i,j] * i / (nbit)**2
#
#     std2 = np.zeros((h,w), dtype=np.float32)
#     for i in range(nbit):
#         for j in range(nbit):
#             std2 += (glcm[i,j] * i - mean)**2
#
#     std = np.sqrt(std2)
#     return std
#
# def fast_glcm_contrast(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm contrast
#     '''
#     h,w = img.shape
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     cont = np.zeros((h,w), dtype=np.float32)
#     for i in range(nbit):
#         for j in range(nbit):
#             cont += glcm[i,j] * (i-j)**2
#
#     return cont
#
# def fast_glcm_dissimilarity(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm dissimilarity
#     '''
#     h,w = img.shape
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     diss = np.zeros((h,w), dtype=np.float32)
#     for i in range(nbit):
#         for j in range(nbit):
#             diss += glcm[i,j] * np.abs(i-j)
#
#     return diss
#
# def fast_glcm_homogeneity(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm homogeneity
#     '''
#     h,w = img.shape
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     homo = np.zeros((h,w), dtype=np.float32)
#     for i in range(nbit):
#         for j in range(nbit):
#             homo += glcm[i,j] / (1.+(i-j)**2)
#
#     return homo
#
# def fast_glcm_ASM(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm asm, energy
#     '''
#     h,w = img.shape
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     asm = np.zeros((h,w), dtype=np.float32)
#     for i in range(nbit):
#         for j in range(nbit):
#             asm  += glcm[i,j]**2
#
#     ene = np.sqrt(asm)
#     return asm, ene
#
# def fast_glcm_max(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm max
#     '''
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     max_  = np.max(glcm, axis=(0,1))
#     return max_
#
# def fast_glcm_entropy(img, vmin=0, vmax=255, nbit=8, ks=5):
#     '''
#     calc glcm entropy
#     '''
#     glcm = fast_glcm(img, vmin, vmax, nbit, ks)
#     pnorm = glcm / np.sum(glcm, axis=(0,1)) + 1./ks**2
#     ent  = np.sum(-pnorm * np.log(pnorm), axis=(0,1))
#     return ent

In [104]:
glcm_mean = fast_glcm_mean(image_gray)
glcm_mean

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Feature

In [105]:
feature = [Hausdorff_D, C, Se, glcm_mean]
feature


[1.9662861686967643,
 0.5984062051758858,
 5.054648392802223,
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]